In [1]:
import sys
!{sys.executable} -m pip install langchain
!{sys.executable} -m pip install openai
!{sys.executable} -m pip install pandas

In [2]:
from langchain.prompts import (
    PromptTemplate,
    FewShotPromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

#import openai
#from langchain import *
#from pydantic import *
import os 
import pandas as pd 

with open("secret.txt") as f:
    key=f.read()
os.environ['OPENAI_API_KEY']=key[:-1]


In [3]:


example_formatter = """sentence: {sentence}
Tag: {tag}"""

example_prompt = PromptTemplate(
    input_variables=["sentence", "tag"],
    template=example_formatter
)

suffix = "sentence: {sentence}\nTag:"

body_prefix = """Classify the following as one of:

1. body: if the description includes the name of a body part (head, back, shoulder, leg, etc)
2. not body: if the description does not mention any body parts """

position_prefix = """Classify the following as one of:

1. position: if the description includes positional language such as right, left, up, over, etc
2. not position: if the description does not mention positions or facing directions """

shape_prefix = """Classify the following as one of:

1. shape: if the description includes a geometric shape word (ex: shape, triangle, angle, point, etc)
2. not shape: if the description does not reference geometric shapes"""

posture_prefix = """Classify the following as one of:

1. posture: if the description is about the posture or action of a person or creature (ex: standing, marching, kicking, etc)
2. not posture: if the description is not about a person or creature's posture"""

abstract_prefix = """Classify the following as one of:

1. abstract: if the description is in terms of a type of person or creature
2. not abstract: if the description is concrete and not metaphorical """

part_prefix = """Classify the following as one of:

1. part: if the description is about part of the image or person or animal
2. not part: if the description is about the overall image, person, or animal """

abstract_examples = [
    {"sentence": "dancer", "tag": "abstract"},
    {"sentence": "monk with big sleeves", "tag": "abstract"},
    {"sentence": "person about to jump off a cliff", "tag": "abstract"},
    {"sentence": "square head on the right", "tag": "not abstract"},
    {"sentence": "looking down", "tag": "not abstract"},
]

posture_examples =[]

body_examples = [
    # {"sentence": "has two heads", "tag": "body"},
    # {"sentence": "with a triangle face", "tag": "body"},
    # {"sentence": "carrying bag on his shoulder", "tag": "body"},
    # {"sentence": "kicking in the air", "tag": "not body"},
    # {"sentence": "dancer", "tag": "not body"},
]

position_examples = [
    # {"sentence": "one foot on the ground", "tag": "position"},
    # {"sentence": "head tilted to the right", "tag": "position"},
    # {"sentence": "diamond on the left", "tag": "position"},
    # {"sentence": "standing", "tag": "not position"},
    # {"sentence": "person about to jump off a cliff", "tag": "not position"},
]

shape_examples = [
    # {"sentence": "diamond on the left", "tag": "shape"},
    # {"sentence": "square head", "tag": "shape"},
    # {"sentence": "weird shape", "tag": "shape"},
    # {"sentence": "head on the left", "tag": "not shape"},
    # {"sentence": "monk with big sleeves", "tag": "not shape"},
]

part_examples = [
    {"sentence": "one foot on the ground", "tag": "part"},
    {"sentence": "one leg raised", "tag": "part"},
    {"sentence": "diamond on the left", "tag": "part"},
    {"sentence": "standing facing left", "tag": "not part"},
    {"sentence": "dancer", "tag": "not part"},
]

def make_chat_template(instruction, examples):
    system_message = SystemMessage(content=instruction)
    messages = []
    for example in examples:
       messages.append(HumanMessage(content=example["sentence"]))
       messages.append(AIMessage(content=example["tag"]))
    messages.append(HumanMessagePromptTemplate.from_template("{test_sentence}"))

    return ChatPromptTemplate.from_messages([system_message, *messages])

body_chat = make_chat_template(body_prefix, body_examples)
position_chat = make_chat_template(position_prefix, position_examples)
shape_chat = make_chat_template(shape_prefix, shape_examples)
abstract_chat = make_chat_template(abstract_prefix, abstract_examples)
part_chat = make_chat_template(part_prefix, part_examples)
posture_chat = make_chat_template(posture_prefix, posture_examples)



    

In [4]:



def tag_data_with_langchain(df, model, prompt): 
  
  res = []
  #for index in range(100):
  for index, _ in df.iterrows():
    print(index)
    #print(model)
    #print(prompt)
    chain=LLMChain(llm=model,prompt=prompt)
    output=chain.run(df["gpt_out"][index])
    print(df["gpt_out"][index])
    #string_output=output.content
    print(output)
    res.append(output)
  return res





In [5]:
model_name = "gpt-3.5-turbo"
#model_name="gpt-4"
temperature = 0
llm = ChatOpenAI(model_name=model_name, max_retries=15, temperature=0)

chain=LLMChain
input =pd.read_csv("data/test/sample_to_tag.csv")

test_input=input
#print(test_input)

body = tag_data_with_langchain(test_input, llm, body_chat)
position = tag_data_with_langchain(test_input, llm, position_chat)
shape= tag_data_with_langchain(test_input, llm, shape_chat)
abstract = tag_data_with_langchain(test_input, llm, abstract_chat)
part=tag_data_with_langchain(test_input, llm, part_chat)
posture=tag_data_with_langchain(test_input, llm, posture_chat)

output=test_input
output['body']=body
output['position']=position
output['shape']=shape
output['abstract']=abstract
output['part']=part
output['posture']=posture

output.to_csv("data/test/outsample_3.csv")


0
facing left
not body
1
head to the right
body
2
over his back
1. body
3
leg out to the left
body
4
his arms are level with his head
body
5
his head comes past the right side of his body by a bit
body
6
the only one with a leg out to the left
body
7
the persons legs are facing the left
body
8
his head which is shaped almost like the box is facing to the right
body
9
walking to the left
not body
10
a bit like the thriller dance
not body
11
a man walking to the left
not body
12
with a square backpack on
not body
13
triangle on his face
not body
14
one leg on floor to the left
body
15
one leg left is slightly raised
body
16
leg lifted to the left
body
17
thriller dance
not body
18
thriller
not body
19
a person dancing
not body
20
one foot on the floor
body
21
another foot is raised
body
22
tilted back
body
23
made from a square shape
not body
24
4 sided shape
not body
25
the last one
not body
26
we havent selected
not body
27
the one with 2 heads almost
not body
28
dancing man
not body
2

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


drunken zombie pose
not body
94
facing left
not body
95
with head tipping back towards the right
1. body
96
one leg in the air
body
97
Drunk zombie
not body
98
with leg slightly in the air
body
99
pointing left
not body
100
drunken zombie
not body
101
looks like a person with two legs
body
102
one is up
not body
103
one down
not body
104
kicking to the left
not body
105
head falling off
body
106
two legs
1. body
107
1 on the floor
not body
108
the other kicking to the left
not body
109
head is off centre
body
110
both legs showing
body
111
one kicking left
body
112
diamond coming out of right side of head
body
113
standing facing left
not body
114
on one foot
body
115
head falling off
body
116
a person kicking one foot in front of them
body
117
Their head looks like it has a beak
body
118
a box over their right shoulder
body
119
looking like a person carrying another person over their shoulder
body
120
michael jackson thriller dance
not body
121
facing left
not body
122
one leg
body
12

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Wed, 11 Oct 2023 17:18:41 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '8148bbfba852980b-SJC', 'alt-svc': 'h3=":443"; ma=86400'}.


one leg up on the left
posture
257
one on the ground
not posture
258
dancing birdman
1. posture
259
facing left
posture
260
big triangle for a head
not posture
261
square balanced over right shoulder
1. posture
262
with one leg up
1. posture
263
Bird man
not posture
264
Square on right
not posture
265
balancing on corner
1. posture
266
Left foot off the ground
posture
267
right foot on
1. posture
268
bird man dancing
1. posture
269
bird man
1. posture
270
Square head
not posture
271
to the top right
not posture
272
Two legs on the left
posture
273
one is lifted up
not posture
274
falling
1. posture
275
the head
not posture
276
falling head
1. posture
277
both legs are on the left
not posture
278
square head
not posture
279
"looks like its falling off"
not posture
280
code name was FALLING
not posture
281
falling head
1. posture
282
two feet facing left
posture
283
walking left
1. posture
284
head on right
not posture
285
falling guy
1. posture
286
walking left
1. posture
287
head on ri